# Выравнивание данных в C++

Это способ размещения данных в оперативной памяти для эффективной работы с ними.

Память можно представить как огромный массив байтов, а адрес - это индекс этого массива.

```
[byte 0] [byte 1] [byte 2] [byte 3]...
```

1 байт адресовать можно как угодно, т. к. это минимальная адресная единица памяти.

Но в C++, помимо типов данных, занимающих 1 байт (char, unsigned char), существует множество более крупных единиц, которые могут хранить информацию в 2-ух, 4-ёх и даже 8-ми байтах.

Считывать эти данные по одному байту не будет эффективно, поэтому данные из оперативной памяти считываются по 1, 2, 4 или 8 байтам в зависимости от того, что нужно считать.

Также максимальный размер машинного слова ограничен архитектурой процессора (разрядностью шины).
Сейчас это 8 байт для 64-разрядных процессоров и 4 байта для 32-разрядных.

Количество считываемых за раз байтов называется **машинным словом**.

**Для того, чтобы лучше понимать, о чём идёт речь, советую посмотреть [это](https://www.youtube.com/watch?v=k9wK2FThEsk), [это](https://www.youtube.com/watch?v=Wh22_O8jXVQ) и [это](https://www.youtube.com/watch?v=7n_8cOBpQrg).**

Если мы работает с данными типа *int*, для хранения которого нужно 4 байта памяти, то данные из оперативной памяти будут считываться группами по 4 байта, а далее обрабатываться в процессоре:

```
[byte 0] [byte 1] [byte 2] [byte 3] [byte 4] [byte 5] [byte 6] [byte 7] [byte 8]...
[          machine word 0         ] [          machine word 1         ]...
```

Считывание начинается с 0-го байта.

**Почему нельзя начать, например, с 1-го байта?**

```
[byte 0] [byte 1] [byte 2] [byte 3] [byte 4] [byte 5] [byte 6] [byte 7] [byte 8]...
         [          machine word 0         ] [          machine word 1         ]...
```

Предположим, что мы начали считать машииные слова с 1-го байта, тогда в 0-ом байте можно разместить только данные, занимающие 1 байт, что не очень эффективно.

Также большинство компьютеров устроено так, что считывать данные с 0-го байта получается быстрее, чем, например, с 1.

К тому же многие архитектуры вовсе не умеют читать слова, которые получаются по шагам от начала, за раз.
Поэтому такой машине нужно сделать два обращения к памяти вместо одного, а после получить из двух слов одно, что займёт много времени:

```
[byte 0] [byte 1] [byte 2] [byte 3] [byte 4] [byte 5] [byte 6] [byte 7] [byte 8]...
         [      machine word to read       ]
[          machine word 0         ] [          machine word 1         ]
```

## Хранение базовых типов данных в памяти

Размер машинного слова зависит от количества памяти, требуемой для какого-либо типа данных.

Например, если нужно работать с типом char, то размер машинного слова будет равен 1-му байту. 
Соответственно, такое машинное слово сможет обратиться к каждому байту в оперативной памяти.

Если нужно работать с типом short, то размер машинного слова будет равен 2-ум байтам. Значит, такое машинное слово сможет обратиться к каждому второму байту и будет считывать по 2 байта: 
```
[byte 0] [byte 1] [byte 2] [byte 3]...
[ machine word 0] [ machine word 1]...
```

Если нужно работать с типом int, то размер машинного слова будет равен 4-ём байтам. Тогда такое машинного слово сможет обратиться к каждому 4-ому байту и будет считывать по 4 байта:
```
[byte 0] [byte 1] [byte 2] [byte 3] [byte 4] [byte 5] [byte 6] [byte 7]...
[          machine word 0         ] [          machine word 1         ]...
```

И т. д.

Узнать количество памяти, необходимое для хранения какого-либо типа данных можно узнать с помощью оператора 
**sizeof()**.

In [23]:
#include <iostream>

using namespace std;

In [24]:
cout << sizeof(char) << endl;
cout << sizeof(short) << endl;
cout << sizeof(int) << endl;

1
2
4


А с помощью оператора **alignof()** можно узнать размер машинного слова для определённого типа данных.

In [25]:
cout << alignof(char) << endl;
cout << alignof(short) << endl;
cout << alignof(int) << endl;

1
2
4


## Хранение структур в памяти

Пусть у нас есть структура CharStruct, в которой содержатся 5 полей типа char.

In [7]:
struct CharStruct {
    char a, b, c, d, e;
}

В памяти эти поля будут записаны друг за другом: 
```
[byte a] [byte b] [byte c] [byte d] [byte e]
[            CharStruct bytes              ]
```

Зная, что char занимает в памяти 1 байт, можно посчитать, что данная структура должна занимать 5 * 1 байт = 5 байт.

Количество памяти, выделяемой под CharStruct:

In [8]:
cout << sizeof(CharStruct);

5

Теперь рассмотрим структуру ShortAndCharStruct, в которой содержатся поля типов short и char.

In [11]:
struct ShortAndCharStruct {
    short a;
    char b;
}

Зная, что short занимает в памяти 2 байта, а char - 1 байт, то путём сложных расчётов можно понять, 
что эта структура данных будет занимать в памяти (2 + 1) байт = 3 байт.

Вызовем оператор *sizeof()* и посмотрим, сколько памяти требуется для хранения данного объекта

In [12]:
cout << sizeof(ShortAndCharStruct);

4

Структуре требуется 4 байта для хранения, а не 3. Почему?

Структура будет считываться с машинным словом в 2 байта.

Если бы мы инициализировали массив, состоящий из нескольких таких структур, 
то, т. к. все элементы массива стоят в памяти друг за другом, усложнилось бы считывание 2, 3 и последующих позиций в массиве, ведь они бы располагались на пересечении двух машинных слов.

Посмотрим, как будет храниться структура в памяти:

```
[byte 0] [byte 1] [byte 2] [byte 3]...
[    short a    ] [char b] [ PAD 0]...
```

Здесь оставшийся 1 байт занимает паддинг (англ. padding - набивка), который считывается, но никак не влияет на обработку данных процессором. 

Если бы это дополнение отсутствовало, то следующие данные в оперативной памяти должны были бы начинаться с 3-го байта, что могло бы привести к проблемам с производительностью при считывании, если бы там лежали данные, занимающие более 1-го байта (случай со "сдвинутым" машинным словом).

Размер машинного слова для структуры ShortAndCharStruct:

In [17]:
cout << alignof(ShortAndCharStruct);

2

**Итог:** 

Размер структуры данных будет кратен размеру её поля, тип данных которого занимает максимальное количество байт по сравнению с типами других полей этой структуры.

### Рубрика "Техномагия и оптимизации"

Рассмотрим структуру MagicStruct, состоящую из двух полей типа char и одного поля типа int.

In [13]:
struct MagicStruct {
    char a;
    char b;
    int c;
}

Зная, что int занимает в памяти 4 байта, а два типа char - 2 байта, получим, 
что, как минимум, это структуре данных нужно 6 байт.

Зная про выравнивание, мы теперь осознаём, что эта структура данных в памяти будет занимать 8 байт.

Проверим это:

In [14]:
cout << sizeof(MagicStruct);

8

А теперь немного изменим нашу структуру и поменяем местами поля *b* и *c*:

In [15]:
struct MagicStructRebuild {
    char a;
    int c;
    char b;
}

С виду структура не изменилась, но посмотрим, сколько памяти они занимает теперь:

In [16]:
cout << sizeof(MagicStructRebuild);

12

**!?!??!?!?!?!?**

Вспоминаем про выравнивание.

Считываться эта структура данных будет с машинным словом в 4 байта, т. к. в ней есть поле типа int.

Поля структуры в памяти записываются друг за другом, соответственно, сначала будет записано поле *a*, потом *c*, потом *b*.

Т. к. поле *int c* должно считаться за одно обращение, то оно будет выравнено.
Но перед ним стоит поле *char a*, которое тоже выравнено, соответственно, 3 байта после *a* будут являться паддингами, потом будет записано поле *c*, а после него поле *b*, которое занимает 1 байт, но, т. к. машинное слово представлено 4-мя байтами, то после поля *c* также будет 3 байта паддинга:

```
[byte 0] [byte 1] [byte 2] [byte 3] [byte 4] [byte 5] [byte 6] [byte 7]
[char a] [ PAD a] [ PAD a] [ PAD a] [              int c              ]
[          machine word 0         ] [          machine word 1         ]


[byte 8] [byte 9] [byte A] [byte B] [byte C] [byte D] [byte E] [byte F]
[char b] [ PAD b] [ PAD b] [ PAD b]
[          machine word 2         ] [          machine word 3         ]
```

**Итог:**

Нужно в правильном порядке выставлять поля в структуре данных, 
т. к. это может повлиять на количество памяти, занимаемой этой структурой.

## Выравнивание при структуре в структуре

Пусть есть структура A с полями типа int и char и структура B с полями типа A и short.

In [18]:
struct A {
    int a;
    char b;
}

In [19]:
struct B {
    A a;
    short b;
}

Размер структуры A равен 8 байтам. 
Выравнивание структуры A - 4 байта.

In [22]:
cout << sizeof(A) << endl;
cout << alignof(A);

8
4

В структуре B поле *a* занимает 8 байт, поле *b* - 2 байта.

Получается, минимальный размер B - 10 байт.

Посмотрим результат sizeof(B):

In [27]:
cout << sizeof(B) << endl;

12


И размер машинного слова структуры B:

In [30]:
cout << alignof(B) << endl;

4


Размер машинного слова равен 4, т. к. максимальный объём памяти из всех полей B и всех полей A - int (A::a).

Соответственно, ближайшее число, которое больше или равно минимального объёма памяти (10), необходимого структуре B, и кратно размеру машинного слова структуры (4), это 12 байт.

В памяти структура B будет храниться так:

```
[byte 0] [byte 1] [byte 2] [byte 3] [byte 4] [byte 5] [byte 6] [byte 7] 
[              int a              ] [char b] [ PAD b] [ PAD b] [ PAD b]
[                              struct A                               ]
[~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~stru-

[byte 8] [byte 9] [byte A] [byte B] [byte C] [byte D] [byte E] [byte F]...
[  short B::b   ] [PAD Bb] [PAD Bb]
[           short B::b            ]
-ct B~~~~~~~~~~~~~~~~~~~~~~~~~~~~~]
```

### Ещё немного техномагии

Если создать такую структуру, которая по полям похожа на предыдущую, но без вложения структуры A:

In [31]:
struct BWithoutA {
    // A begin
    int a;
    char b;
    // A end
    short B_b;
}

И вывести её размер:

In [32]:
cout << sizeof(BWithoutA) << endl;

8


То получится 8 байт, т. к. в памяти она будет храниться так:

```
[byte 0] [byte 1] [byte 2] [byte 3] [byte 4] [byte 5] [byte 6] [byte 7]...
[              int a              ] [char b] [ PAD b] [   short B_b   ]
```

## Выравнивание при наследовании

Теперь у нас есть структура B с полем short c, которая наследуется от структуры A с полями int a и char b:

In [33]:
struct A {
    int a;
    char b;
}

In [34]:
struct B : A {
    short c;
}

Размер и размер машинного слова B:

In [35]:
cout << sizeof(B) << endl;
cout << alignof(B) << endl;

12
4


Получилось то же самое, что и при вложении структуры A в структуру B.

При наследовании сначала выделяется память для структуры-родителя, после для полей самой структуры.

При множественном наследовании ситуация аналогична: сначала выделяется память для структур-родителей в порядке их указания, после для полей самой струтуры:

```
[byte 0] ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... [byte N]
[struct-parent 0] ... [struct-parent P] [struct-field 0] ... [struct field M]
```